In [ ]:
from time import time

import numpy as np
import matplotlib.pyplot as plt

import mnist
from sklearn.decomposition import PCA

from ipywidgets import interact, interactive
import ipywidgets as widgets

from pathlib import Path

In [ ]:
import time
from pathlib import Path

tic=time.time()

# Asegure que el directorio de datos existe.
datadir='./mnist_data/'
Path(datadir).mkdir(parents=False, exist_ok=True)

# temporary_dir es una función que retorna el directorio temporal;
mnist.temporary_dir = lambda: datadir

# Prepare los datos de entrenamiento
train_images = mnist.train_images() # Esto es un tensor m x 28 x 28
train_labels = mnist.train_labels().astype(int)

m,h,w=train_images.shape
train_data = train_images.reshape((m,w*h)).astype(float)/255 # Esto es una matrix de m x 784

In [ ]:
X=np.array(train_data)
y=np.array(train_labels)

_, axes = plt.subplots(1, 4)
images_and_labels = list(zip(X, y))
for ax, (image, label) in zip(axes, images_and_labels[:4]):
    ax.set_axis_off()
    ax.imshow(image.reshape(28,28), cmap=plt.cm.gray, interpolation='nearest')
    ax.set_title('Training: %s' % label)

In [ ]:
n_components = 150

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X.shape[0]))
t0 = time.time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=False).fit(X)
print("done in %0.3fs" % (time.time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

In [ ]:
print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time.time()
X_pca = pca.transform(X)
print("done in %0.3fs" % (time.time() - t0))

In [ ]:
plt.imshow(pca.mean_.reshape(h,w),cmap=plt.cm.gray)

In [ ]:
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())
        
eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

plt.show()

In [ ]:
@interact(c=widgets.IntSlider(min=0,max=50,step=1,value=0,continuous_update=False), 
          f=widgets.FloatSlider(min=-3,max=3,step=0.1,value=0,continuous_update=True))
def eigenmnist(c,f):
    preimage=pca.mean_ + f*np.sqrt(pca.explained_variance_[c])*pca.components_[c]
    plt.imshow(preimage.reshape(28,28),cmap='gray')
    

In [ ]:
## TO-DO: Plot the spectrum (accumulated variance vs # of components)

In [ ]:
## Recalculando, para tener el espectro completo:

t0 = time.time()
pca = PCA(n_components=h*w, svd_solver='randomized',
          whiten=False).fit(X)
print("done in %0.3fs" % (time.time() - t0))

In [ ]:
spectrum=np.cumsum(pca.explained_variance_)
spectrum/=spectrum[-1]
plt.plot(spectrum)

In [ ]:
## TO-DO: Given a set of components, sliders and a button to randomly select a digit, 
## compute the principal components and reconstruct the digit, but allow with the 
## sliders to change the components.

In [ ]:
components.shape

In [ ]:
digit=X[np.random.randint(0,m)]
components=pca.transform(digit.reshape(1,-1)).ravel()
plt.imshow(digit.reshape(h,w),cmap='gray')
f=np.zeros((1,5))
    
# Calcule componentes del dígito excepto los eigenvectores que se estarán modificando
base=pca.mean_ + np.matmul(components.ravel()[5:-1],pca.components_[5:-1])

In [ ]:
@interact(f0=widgets.FloatSlider(min=-3,max=3,step=0.1,
                                 value=components[0]/np.sqrt(pca.explained_variance_[0]),continuous_update=True),
          f1=widgets.FloatSlider(min=-3,max=3,step=0.1,
                                 value=components[1]/np.sqrt(pca.explained_variance_[1]),continuous_update=True),
          f2=widgets.FloatSlider(min=-3,max=3,step=0.1,
                                 value=components[2]/np.sqrt(pca.explained_variance_[2]),continuous_update=True),
          f3=widgets.FloatSlider(min=-3,max=3,step=0.1,
                                 value=components[3]/np.sqrt(pca.explained_variance_[3]),continuous_update=True),
          f4=widgets.FloatSlider(min=-3,max=3,step=0.1,
                                 value=components[4]/np.sqrt(pca.explained_variance_[4]),continuous_update=True))
def eigenmnist(f0,f1,f2,f3,f4):
    preimage=(base + f0*np.sqrt(pca.explained_variance_[0])*pca.components_[0] +
                     f1*np.sqrt(pca.explained_variance_[1])*pca.components_[1] +
                     f2*np.sqrt(pca.explained_variance_[2])*pca.components_[2] +
                     f3*np.sqrt(pca.explained_variance_[3])*pca.components_[3] +
                     f4*np.sqrt(pca.explained_variance_[4])*pca.components_[4] )
    plt.imshow(preimage.reshape(28,28),cmap='gray')